In [96]:
#Dependencies
from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
import pandas as pd
from scipy.io import arff

In [97]:
X_full = pd.read_csv('/workspaces/2025SE-Mike.N-HSC-AT2/data/concrete_engineered.csv')
y = X_full['strength']
X_full = X_full.drop(['strength'], axis = 1)

In [98]:
# Select categorical columns
categorical_cols = [cname for cname in X_full.columns if
                    X_full[cname].nunique() < 10 and 
                    X_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_full.columns if 
                X_full[cname].dtype in ['int64', 'float64']]

features = categorical_cols + numerical_cols
print(categorical_cols, numerical_cols)


[] ['cement', 'blast_furnace_slag', 'fly_ash', 'water', 'superplasticizer', 'coarse_aggregate', 'fine_aggregate', 'age', 'cement_to_water_ratio', 'superplasticizer_to_water_ratio', 'fine_aggregate_to_water_ratio', 'cement_to_fly_ash_ratio', 'superplasticizer_to_fly_ash_ratio', 'age_squared']


No categorical features, so every feature gets treated in the same manner.

In [99]:
X_full.describe()

,cement,blast_furnace_slag,fly_ash,water,superplasticizer,coarse_aggregate,fine_aggregate,age,cement_to_water_ratio,superplasticizer_to_water_ratio,fine_aggregate_to_water_ratio,cement_to_fly_ash_ratio,superplasticizer_to_fly_ash_ratio,age_squared
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,657.000000,1030.000000
mean,281.165631,73.895485,54.187136,181.566359,6.203112,972.918592,773.578883,45.662136,1.578275,0.037402,4.344763,1.074774,0.059260,6071.594175
std,104.507142,86.279104,63.996469,21.355567,5.973492,77.753818,80.175427,63.169912,0.648105,0.039131,0.824908,1.665946,0.062538,20195.954706
min,102.000000,0.000000,0.000000,121.750000,0.000000,801.000000,594.000000,1.000000,0.531250,0.000000,2.605263,0.000000,0.000000,1.000000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,1.069502,0.000000,3.888350,0.000000,0.000000,49.000000
50%,272.900000,22.000000,0.000000,185.000000,6.350000,968.000000,779.510000,28.000000,1.480718,0.034663,4.299479,0.000000,0.057791,784.000000
75%,350.000000,142.950000,118.270000,192.000000,10.160000,1029.400000,824.000000,56.000000,1.875000,0.059592,4.791273,1.905525,0.083333,3136.000000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,3.746827,0.233672,7.840442,11.332379,0.457482,133225.000000


In [100]:
#Check for missing values
X_full.isnull().sum()

cement                                 0
blast_furnace_slag                     0
fly_ash                                0
water                                  0
superplasticizer                       0
coarse_aggregate                       0
fine_aggregate                         0
age                                    0
cement_to_water_ratio                  0
superplasticizer_to_water_ratio        0
fine_aggregate_to_water_ratio          0
cement_to_fly_ash_ratio                0
superplasticizer_to_fly_ash_ratio    373
age_squared                            0
dtype: int64

No missing values.

In [101]:
# Check for duplicates
X_full.duplicated().sum()

34

In [102]:
# Remove duplicates
X_full = X_full.drop_duplicates()
X_full.duplicated().sum()

0

In [103]:
# Preprocessing for numerical data
# RobustScaler is used to scale the data, as it is less prone to outliers
scaler = RobustScaler()
# Imputer is used to fill in missing values. We don't treat 0s as missing values.
imputer = SimpleImputer(strategy='mean')
preprocessing = Pipeline(steps = [('imputer', imputer), ('scaler', scaler)])
X_full = preprocessing.fit_transform(X_full)
X_full = pd.DataFrame(X_full, columns = features)

In [104]:
X_full.describe()

,cement,blast_furnace_slag,fly_ash,water,superplasticizer,coarse_aggregate,fine_aggregate,age,cement_to_water_ratio,superplasticizer_to_water_ratio,fine_aggregate_to_water_ratio,cement_to_fly_ash_ratio,superplasticizer_to_fly_ash_ratio,age_squared
count,996.000000,996.000000,996.000000,996.000000,996.000000,996.000000,996.000000,996.000000,996.000000,996.000000,996.000000,996.000000,9.960000e+02,996.000000
mean,0.111817,0.367240,0.466334,-0.131650,-0.001606,0.061943,-0.074572,0.430364,0.129747,0.078112,0.056013,0.566020,-3.566982e-17,1.845082
std,0.701311,0.606037,0.543051,0.821784,0.592474,0.734861,0.838592,1.522640,0.833108,0.687152,0.977871,0.869841,2.454434e+00,6.978773
min,-1.070695,-0.140351,0.000000,-2.465305,-0.600000,-1.575472,-1.935484,-0.642857,-1.202136,-0.558941,-1.983098,0.000000,-3.018816e+00,-0.266327
25%,-0.473915,-0.140351,0.000000,-0.720894,-0.600000,-0.339623,-0.542144,-0.333333,-0.519353,-0.558941,-0.469237,0.000000,-7.017190e-01,-0.200000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000
75%,0.526085,0.859649,1.000000,0.279106,0.400000,0.660377,0.457856,0.666667,0.480647,0.441059,0.530763,1.000000,2.982810e-01,0.800000
max,1.879694,2.381754,1.691891,2.363146,2.620000,1.669811,2.212279,8.023810,3.004077,3.579911,4.230695,5.947115,1.963861e+01,45.047959


In [105]:
#Add the target column back into the dataset
X_full['strength'] = y
X_full.describe()

,cement,blast_furnace_slag,fly_ash,water,superplasticizer,coarse_aggregate,fine_aggregate,age,cement_to_water_ratio,superplasticizer_to_water_ratio,fine_aggregate_to_water_ratio,cement_to_fly_ash_ratio,superplasticizer_to_fly_ash_ratio,age_squared,strength
count,996.000000,996.000000,996.000000,996.000000,996.000000,996.000000,996.000000,996.000000,996.000000,996.000000,996.000000,996.000000,9.960000e+02,996.000000,996.000000
mean,0.111817,0.367240,0.466334,-0.131650,-0.001606,0.061943,-0.074572,0.430364,0.129747,0.078112,0.056013,0.566020,-3.566982e-17,1.845082,35.721553
std,0.701311,0.606037,0.543051,0.821784,0.592474,0.734861,0.838592,1.522640,0.833108,0.687152,0.977871,0.869841,2.454434e+00,6.978773,16.843789
min,-1.070695,-0.140351,0.000000,-2.465305,-0.600000,-1.575472,-1.935484,-0.642857,-1.202136,-0.558941,-1.983098,0.000000,-3.018816e+00,-0.266327,2.331808
25%,-0.473915,-0.140351,0.000000,-0.720894,-0.600000,-0.339623,-0.542144,-0.333333,-0.519353,-0.558941,-0.469237,0.000000,-7.017190e-01,-0.200000,23.522163
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,33.955659
75%,0.526085,0.859649,1.000000,0.279106,0.400000,0.660377,0.457856,0.666667,0.480647,0.441059,0.530763,1.000000,2.982810e-01,0.800000,46.230572
max,1.879694,2.381754,1.691891,2.363146,2.620000,1.669811,2.212279,8.023810,3.004077,3.579911,4.230695,5.947115,1.963861e+01,45.047959,82.599225


In [106]:
#Save the preprocessed data
X_full.to_csv('/workspaces/2025SE-Mike.N-HSC-AT2/data/concrete_preprocessed.csv', index = False)